In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="기간"),
    ResponseSchema(name="budget", description="예산"),
    ResponseSchema(name="rating", description="추천도 (1-5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()


template = """
여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동을 구조화된 형태로 추출하세요
주요 활동은 리스트형태로 출력
 {plan}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)


model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)


plan = """
  지난 주에 부산으로 2박 3일 여행을 다녀왔어요.
   총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요.
   정말 만족스러운 여행이었습니다. 
   5점 만점에 4점 정도 줄 수 있을 것 같아요."""



chain = prompt | model | parser

output = chain.invoke({"plan": plan})

print(output)

{'destination': '부산', 'duration': '2박 3일', 'budget': '30만원', 'rating': '4', 'activities': '해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경'}
